In [ ]:
import numpy as np
import math 
import torch 
import torch.nn as nn
import scipy.io as sio 

In [5]:


#input_dim = TrainX.shape[1]
hidden_dim = 10
n_layers = 1

seq_len = 1
# IDEA /  try one-hot encoding see if it improves anything 

# check if GPU is available, (useful when we'll have access to the lab's GPU)

# build the class for the model of NN 
class LSTM(nn.Module):
    
    def __init__(self, TrainX, TrainY, n_hidden= 10 ,n_layers = 1, lr=0.001): # no dropout for now 
        super().__init__()
        self.n_hidden = n_hidden
        self.lr = lr
        self.n_layers = n_layers
        self.input_dim = TrainX.shape[1]

        """self.net = nn.Sequential(nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True), 
                         nn.LSTM(hidden_dim, hidden_dim, n_layers, batch_first=True), 
                        nn.Linear(n_hidden, (TrainY.shape[1])))"""
        #lstm layers
        self.lstm = nn.LSTM(self.input_dim, hidden_dim, n_layers, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, n_layers, batch_first=True)
        #output layer
        self.fc = nn.Linear(n_hidden, (TrainY.shape[1]))
    
    def binarize_weights(self, ind_layer) : 
        weights = self.net[ind_layer].weight_ih_l[0] 
        for w in weights : 
            if w >= 0 : 
                w = 1
            else : 
                w = -1 
        self.net[ind_layer].weight_ih_l[k]  = weights 

    
    def forward(self, TrainX, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        ## Get the outputs and the new hidden state from the lstm
        TrainX = TrainX.reshape([seq_len, TrainX.shape[0],TrainX.shape[1]])
        #self.binarize_weights(0)
        r_output, hidden = self.lstm(TrainX, hidden)
        #self.binarize_weights(1)
        r_output, hidden = self.lstm2(r_output, hidden)
        # Stack up LSTM outputs using view
        # reshape the output
        out = r_output.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        #self.binarize_weights(2)
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        hidden_state = torch.randn(n_layers, 1, hidden_dim)
        cell_state = torch.randn(n_layers, 1, hidden_dim)
        hidden = (hidden_state, cell_state)

        return hidden

In [8]:
def train(TrainX, TrainY, net, lossfunc, optimizer, num_epoch, clip = 5):

    for epoch in range(num_epoch):
        # TODO: Step 1 - create torch variables corresponding to features and labels
        
        x = TrainX.reshape([seq_len, TrainX.shape[0],TrainX.shape[1]])
        x = torch.from_numpy(TrainX).float()
        y = torch.from_numpy(TrainY).float()
        
        # initialize hidden state 
        h = net.init_hidden()
        # TODO: Step 2 - compute model predictions and loss
        pred, h = net(x, h)
        
        #target = torch.reshape(y, (-1,)).long()
        loss = lossfunc(pred, y)
        # TODO: Step 3 - do a backward pass and a gradient update step
        optimizer.zero_grad()
        loss.backward()
        # gradient clipping - prevents gradient explosion 
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        if epoch == num_epoch-1:
            #print ('Epoch [%d/%d], Loss: %.4f' %(epoch+1, num_epoch, loss.item()))
            #vec = torch.cat((y,pred),1)
            #vec = vec.detach().numpy()
            corrcoef = np.corrcoef(pred.detach().numpy().reshape((-1,)),y.detach().numpy().reshape((-1,)))
            print ('Epoch [%d/%d], Loss: %.4f' %(epoch+1, num_epoch, loss.item()))
            print ('Correlation coefficient : {corrcoef}'.format(corrcoef=corrcoef))


In [9]:
# Train model 

n_hidden = 10
n_layers = 1

n_epochs =  10 # start small 

#input_dim = TrainX.shape[1]
# train the model
Finger = [0,1,2,3,4]
Idx_subject = 10


def create_Seq_data_set(data_set, look_back):
    data_x = []
    data_y = []
    for i in range(len(data_set - look_back-1)):
        a = data_set[i:(i+look_back),0]
        data_x.append(a)
        data_y.append(data_set[i+look_back,0])
    return np.array(data_x), np.array(data_y)
        
#prepare train and test data
for Finger in list([0,1,2,3,4]): # 5 fingers for each subject. 0:thumb, 1:index, 2:middle ...
    matData = sio.loadmat('data/BCImoreData_Subj_'+str(Idx_subject)+'_200msLMP.mat')
    TrainX = matData['Data_Feature'].transpose()
    TrainY = matData['SmoothedFinger']
    TrainY = TrainY [:,Finger]
    TrainY = TrainY.reshape(TrainY.shape[0],1)
    #load testing data (TestX: feature vectors, TestY: labels)
    matData = sio.loadmat('data/BCImoreData_Subj_'+str(Idx_subject)+'_200msLMPTest.mat')
    TestX = matData['Data_Feature'].transpose()
    TestY = matData['SmoothedFinger']
    TestY = TestY[:,Finger]
    TestY = TestY.reshape(TestY.shape[0],1)

    print("run for finger ", Finger)

    net = LSTM(TrainX, TrainY,  n_hidden, n_layers)

    lossfunc = nn.L1Loss()
    #lossfunc = nn.NLLLoss()
    optimizer = torch.optim.Adamax(net.parameters())
    train(TrainX, TrainY, net, lossfunc, optimizer, n_epochs, clip = 5)
    


run for finger  0
Epoch [10/10], Loss: 0.2341
Correlation coefficient : [[ 1.         -0.00686595]
 [-0.00686595  1.        ]]
run for finger  1
Epoch [10/10], Loss: 0.5660
Correlation coefficient : [[1.         0.00342156]
 [0.00342156 1.        ]]
run for finger  2
Epoch [10/10], Loss: 0.3955
Correlation coefficient : [[ 1.         -0.00285423]
 [-0.00285423  1.        ]]
run for finger  3
Epoch [10/10], Loss: 0.2555
Correlation coefficient : [[ 1.         -0.04768108]
 [-0.04768108  1.        ]]
run for finger  4
Epoch [10/10], Loss: 0.3139
Correlation coefficient : [[1.         0.02943972]
 [0.02943972 1.        ]]
